In [1]:
import matplotlib.pyplot as plt
import config
from CA import *

In [ ]:
y = [0]*50
x=[0]*10
N=5000
# test for 50 different reaction time values between 0 and 10
for i in range(50):
    for j in range(10):
        cars, road = main_loop(P_init=0.5, iterations=N, road_len=1000,
                                Vmax1=5, Vmax2=4, Vrandom=True,
                                 p_gen=0.5, reaction_time=i, p_Vrandom=0.5,
                                Vrandom_amount=1, Vm_vary=0)
        x[j] = config.flow_counter
    y[i] = sum(x)/10
plt.plot(np.linspace(0,50,50),
        y)